List the categories, glasses, ingredients or alcoholic filters
www.thecocktaildb.com/api/json/v1/1/list.php?c=list
www.thecocktaildb.com/api/json/v1/1/list.php?g=list
www.thecocktaildb.com/api/json/v1/1/list.php?i=list
www.thecocktaildb.com/api/json/v1/1/list.php?a=list

Filter by Category [There are 11 categories]
www.thecocktaildb.com/api/json/v1/1/filter.php?c=Ordinary_Drink
www.thecocktaildb.com/api/json/v1/1/filter.php?c=Cocktail

Filter by Glass [There are 32 glasses]
www.thecocktaildb.com/api/json/v1/1/filter.php?g=Cocktail_glass
www.thecocktaildb.com/api/json/v1/1/filter.php?g=Champagne_flute

Filter by ingredient [There are 100 ingredients]
www.thecocktaildb.com/api/json/v1/1/filter.php?i=Gin
www.thecocktaildb.com/api/json/v1/1/filter.php?i=Vodka

Filter by alcoholic [There are 3 alcohol types]
www.thecocktaildb.com/api/json/v1/1/filter.php?a=Alcoholic
www.thecocktaildb.com/api/json/v1/1/filter.php?a=Non_Alcoholic

Search cocktail by name [It's similar to ingredients but more fields]
www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita

Search cocktails by first letter  [It's similar to ingredients but more fields]
www.thecocktaildb.com/api/json/v1/1/search.php?f=a

In [68]:
import pandas as pd
import mysql.connector
import sqlalchemy
from sqlalchemy import  text, MetaData
import sqlite3
import requests
import json
import seaborn as sns
from matplotlib import pyplot as plt

# parameter in Azure database/ test
userName = 'ravi'
userPass = 'Password507'
host='sakila-2.mysql.database.azure.com'
port=3306
database='test'

In [17]:
# list of category(c), glass(g), ingredient(i), alcohol(a)
#response_API = requests.get("https://www.thecocktaildb.com/api/json/v1/1/list.php?a=list") 

#Search cocktails by first letter  [It's similar to ingredients but more fields]
response_a_API = requests.get("https://www.thecocktaildb.com/api/json/v1/1/search.php?f=a")
response_b_API = requests.get("https://www.thecocktaildb.com/api/json/v1/1/search.php?f=b")
response_c_API = requests.get("https://www.thecocktaildb.com/api/json/v1/1/search.php?f=c")

In [18]:
# Return parsed JSON data
data_a = response_a_API.json()
data_b = response_b_API.json()
data_c = response_c_API.json()

# Convert the combined data to a DataFrame
df = pd.DataFrame(data_a['drinks']+ data_b['drinks']+ data_c['drinks'])

# Display the DataFrame
#df.head(10)

In [69]:
# create engine to execute 
engine = sqlalchemy.create_engine(f"mysql+mysqlconnector://{userName}:{userPass}@{host}:{port}/{database}")

#check if connection is successfull
"""
try:
    connection = engine.connect()
    print("Connection successful!")
    connection.close()
except Exception as e:
    print(f"Error: {e}")

test_tables = pd.read_sql_query("Show Tables", engine)

# Print the DataFrame
print(test_tables)
"""


'\ntry:\n    connection = engine.connect()\n    print("Connection successful!")\n    connection.close()\nexcept Exception as e:\n    print(f"Error: {e}")\n\ntest_tables = pd.read_sql_query("Show Tables", engine)\n\n# Print the DataFrame\nprint(test_tables)\n'

In [20]:
table_name = 'cocktail'

# Load DataFrame into Azure MySQL table
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

75

In [75]:
import tkinter as tk
from tkinter import simpledialog

global_user_input = None
global_option = None

def show_popup(option):
    global global_user_input, global_option
    
    # Ask the user for input with a pop-up window
    user_input = simpledialog.askstring("Input", f"Enter {option}:")
    
    # Check if the user clicked Cancel or entered a value
    if user_input is not None:
        result_label.config(text=f"User searched for {option}: {user_input}")
        global_user_input = user_input
        global_option = option
        print(f"User searched for {option}: {user_input}")
        root.destroy()

    else:
        result_label.config(text=f"User clicked Cancel for {option}")

# Create the main window
root = tk.Tk()
root.title("Button Selection")

# Create buttons to trigger the pop-up for "Glass" and "Ingredient"
glass_button = tk.Button(root, text="Glass", command=lambda: show_popup("Glass"))
glass_button.pack(pady=10)

ingredient_button = tk.Button(root, text="Ingredient", command=lambda: show_popup("Ingredient"))
ingredient_button.pack(pady=10)

# Create a label to display the result
result_label = tk.Label(root, text="")
result_label.pack()

# Start the Tkinter event loop
root.mainloop()

# Access global_user_input and global_option outside the Tkinter event loop
if global_user_input is not None and global_option is not None:
    print("option:", global_option)
    print("user_input:", global_user_input)


User searched for Glass: shot
option: Glass
user_input: shot


In [76]:
#Query Cocktail Table by Ingridients, Glass, Category, Alcohol
if global_option == "Ingredient":
    count_query = text(f"SELECT count(*) FROM cocktail WHERE strIngredient1 like '%{global_user_input}'")
    select_query = text(f"""SELECT 
                        strDrink, strCategory, strGlass, strAlcoholic, strIngredient1, strInstructions
                        FROM cocktail WHERE strIngredient1 like '%{global_user_input}'""")
else:
    global_user_input = global_user_input+" glass"
    count_query = text(f"SELECT count(*) FROM cocktail WHERE strglass = '{global_user_input}'")
    select_query = text(f"""SELECT
                        strDrink, strCategory, strGlass, strAlcoholic, strIngredient1, strInstructions  
                        FROM cocktail WHERE strglass = '{global_user_input}'""")

# Execute the SELECT query
with engine.connect() as conn:
    count = conn.execute(count_query)
    count_result = count.scalar()
    
    result = conn.execute(select_query)
    rows = result.fetchall()

#Display the result
print(f"Search count: {count_result}")
df_query = pd.DataFrame(rows, columns=result.keys())
df_query

Search count: 8


,strDrink,strCategory,strGlass,strAlcoholic,strIngredient1,strInstructions
0,ABC,Shot,Shot glass,Alcoholic,Amaretto,Layered in a shot glass.
1,ACID,Shot,Shot glass,Alcoholic,151 proof rum,Poor in the 151 first followed by the 101 serv...
2,B-52,Shot,Shot glass,Alcoholic,Baileys irish cream,Layer ingredients into a shot glass. Serve wit...
3,Big Red,Shot,Shot glass,Alcoholic,Irish cream,Pour ingredients into 1 ounce shot glass
4,Bob Marley,Shot,Shot glass,Alcoholic,Midori melon liqueur,Layer in a 2 oz shot glass or pony glass
5,Bubble Gum,Shot,Shot glass,Alcoholic,Vodka,Layer in order into a shot glass.
6,Bumble Bee,Shot,Shot glass,Alcoholic,Baileys irish cream,This is a layered shot. First pour the Bailey'...
7,Chocolate Milk,Shot,Shot Glass,Alcoholic,Chocolate liqueur,"Put the milk in the bottom, pour the Liquer on..."


In [23]:
"""# Close the conn, engine
conn.close()
engine.dispose()"""

'# Close the conn, engine\nconn.close()\nengine.dispose()'

In [ ]:
#Plot